In [ ]:
############################################################################################################################################
# !!!   Enter in your Ngrok authentication token below between the "". You'll have to sign up on Ngrok if you don't already have one (it's free btw)   !!! 
############################################################################################################################################

NGROK_AUTH_TOKEN=""   # Dummy Example:"2X1rY0t8SjDwqaG0pbYUtkt8ki_K8xepLG9ZUg5Wjp0HAAA"
USER_PASSWORD="password"
USER_NAME="user"

In [ ]:
# Import
import os

# Create user
os.system(f'sudo adduser --disabled-password --gecos "" {USER_NAME}')
os.system(f"echo '{USER_NAME}:{USER_PASSWORD}' | sudo chpasswd")

# Install and start ssh-server
!sudo apt install -y openssh-server
!sudo service ssh status
!sudo service ssh start
!sudo service ssh status

# Install ngrok
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update -y && sudo apt install -y ngrok

# Reset terminal color
!echo -e "\033[0m"
    
# Run ngrok and pass logs into ngrok.log. When Ngrok tunnel is created, show the ssh command for user to run
os.system(f"""ngrok tcp 22 --authtoken {NGROK_AUTH_TOKEN}""" + r""" --log=stdout | tee ngrok.log | grep --line-buffered -oE 'tcp://[^:]*:[0-9]+' | sed -u 's/$/\n\nTHIS IS YOUR PASSWORD:\n"""+ USER_PASSWORD +r"""/' | sed -u 's/:/ \-p /2' | sed -u 's/\/\//\nssh """+ USER_NAME + r"""\@/' | sed -u 's/tcp\:/\n\nEXECUTE THE BELOW COMMAND IN YOUR LOCAL TERMINAL: /'""")